# Fine-tuning GPT-2

En este notebook, utilizaremos un "Supervised Learning" para hacer que el modelo pueda cumplir instrucciones más concretas.

En esencia, utilizaremos cada ejemplo como una unidad autónoma (con potencial para algo llamado "empaquetado") y esto nos permitirá construir datos "etiquetados".

Lo que buscamos en este entrenamiento es conseguir que un modelo fundacional como GPT-2 sea capaz de únicamente traducir lenguaje natural a SQL.



In [ ]:
!pip install transformers accelerate datasets trl bitsandbytes -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Preparación de los datos

Lo primero que necesitamos es un conjunto de datos que nos ayude a entrenar el modelo con esta instrucción. Tenemos este conjunto de datos por ejemplo: [sql-create-context](https://huggingface.co/datasets/b-mc2/sql-create-context)


In [ ]:
from datasets import load_dataset

sql_dataset = load_dataset("b-mc2/sql-create-context")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
sql_dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

In [ ]:
sql_dataset['train'][0]

{'answer': 'SELECT COUNT(*) FROM head WHERE age > 56',
 'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)'}

Entonces, tenemos aproximadamente 78,5 mil filas de:

- `question` - una consulta en lenguaje natural sobre
- `context` - la orden `CREATE TABLE` - que nos ofrece un contexto importante sobre la tabla
- `answer` - una consulta SQL que está alineada tanto con la pregunta como con el contexto.

Como en todo entrenamiento de ML vamos a separar el conjunto de datos en `train`, `val` y `test`.

In [ ]:
sql_dataset_train_test = sql_dataset["train"].train_test_split(test_size=0.2)

In [ ]:
sql_dataset_train_test

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62861
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 15716
    })
})

In [ ]:
sql_dataset_val_test = sql_dataset_train_test["test"].train_test_split(test_size=0.5)

In [ ]:
sql_dataset_val_test

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})

In [ ]:
from datasets import DatasetDict

split_sql_dataset = DatasetDict({
    "train" : sql_dataset_train_test["train"],
    "val" : sql_dataset_val_test["train"],
    "test" : sql_dataset_val_test["test"]
})

In [ ]:
split_sql_dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 62861
    })
    val: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 7858
    })
})

### Creando el "Prompt" de entrenamiento

Ahora necesitamos crear un prompt que nos permita interactuar con nuestro modelo y conseguir el comportamiento entrenado.

El objetivo de este prompt será ofrecerle como un patrón que alinea el modelo con nuestros resultados deseados.

La idea básica es que fusionaremos la pregunta, el contexto, y la respuesta en un solo bloque de texto que muestre al modelo nuestros resultados deseados.


```
{bos_token}

### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}

{eos_token}
```

Ejemplo:

```
<|startoftext|>

### Instruction:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.
You must output the SQL query that answers the question.

### Input:
How many locations did the team play at on week 7?

### Context:
CREATE TABLE table_24123547_2 (location VARCHAR, week VARCHAR)

### Response:
SELECT COUNT(location) FROM table_24123547_2 WHERE week = 7

<|endoftext|>
```

Lo que haremos es "modificar" nuestro conjunto de datos creando un prompt como este para cada registro para poder entrenar nuestro modelo.


### Plantilla de Prompt:

Crearemos dos prompts, uno para el entrenamiento y otro para hacer inferencia. La diferencia es que en el de inferencia no pondremos la respuesta.


In [ ]:
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:
{response}{eos_token}
"""

TEXT2SQL_INFERENCE_PROMPT_TEMPLATE = """\
{bos_token}### Instruction:
{system_message}

### Input:
{input}

### Context:
{context}

### Response:{eos_token}
"""

### System Message:

Necesitamos crear un `SYSTEM_MESSAGE`. Esta es la instrucción básica con el objetivo a realizar del prompt.

In [ ]:
SYSTEM_MESSAGE = "You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question."

In [ ]:
def create_sql_prompt(sample):
  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"],
      response = sample["answer"]
  )

  return {"text" : full_prompt}

In [ ]:
def create_sql_prompt_and_response(sample):
  full_prompt = TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
      bos_token = "<|startoftext|>",
      eos_token = "<|endoftext|>",
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"]
  )

  ground_truth = sample["answer"]

  return {"full_prompt" : full_prompt, "ground_truth" : ground_truth}

In [ ]:
split_sql_dataset["train"][0]

{'answer': 'SELECT MIN(weeks_at__number1) FROM table_27441210_5 WHERE year = 1988',
 'question': 'Name the least weeks at number 1 for 1988',
 'context': 'CREATE TABLE table_27441210_5 (weeks_at__number1 INTEGER, year VARCHAR)'}

In [ ]:
create_sql_prompt(split_sql_dataset["train"][0])

{'text': '<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n\n### Input:\nName the least weeks at number 1 for 1988\n\n### Context:\nCREATE TABLE table_27441210_5 (weeks_at__number1 INTEGER, year VARCHAR)\n\n### Response:\nSELECT MIN(weeks_at__number1) FROM table_27441210_5 WHERE year = 1988<|endoftext|>\n'}

Lo aplicamos a todo el dataset

In [ ]:
split_sql_dataset = split_sql_dataset.map(create_sql_prompt)

Map:   0%|          | 0/62861 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

Map:   0%|          | 0/7858 [00:00<?, ? examples/s]

In [ ]:
split_sql_dataset['train'][0]

{'answer': 'SELECT MIN(weeks_at__number1) FROM table_27441210_5 WHERE year = 1988',
 'question': 'Name the least weeks at number 1 for 1988',
 'context': 'CREATE TABLE table_27441210_5 (weeks_at__number1 INTEGER, year VARCHAR)',
 'text': '<|startoftext|>### Instruction:\nYou are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.\n\n### Input:\nName the least weeks at number 1 for 1988\n\n### Context:\nCREATE TABLE table_27441210_5 (weeks_at__number1 INTEGER, year VARCHAR)\n\n### Response:\nSELECT MIN(weeks_at__number1) FROM table_27441210_5 WHERE year = 1988<|endoftext|>\n'}

<|startoftext|>

### Instruction:
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. You must output the SQL query that answers the question.

### Input:
What is Name, when Venue is "Motherwell"?

### Context:
CREATE TABLE table_name_71 (name VARCHAR, venue VARCHAR)

### Response:
SELECT name FROM table_name_71 WHERE venue = "motherwell"

<|endoftext|>

## Cargar el modelo a afinar y preprocesarlo

Utilizaremos las librerías de Hugging Face! 🤗

- [`AutoModelForCausalLM`](https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/auto#transformers.AutoModelForCausalLM)
- [`AutoTokenizer`](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoTokenizer)
- [GPT-2 Model Card](https://huggingface.co/gpt2)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "gpt2"

gpt2_base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)

gpt2_tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Necesitamos asegurarnos de que nuestro tokenizador tenga un pad_token para poder rellenar las secuencias y que todas tengan la misma longitud.

Utilizaremos un pequeño truco aquí para establecer nuestro token de relleno como nuestro token eos (final de secuencia) para hacer que el entrenamiento sea un poco más fluido.

In [ ]:
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

También tenemos que igualar el tamaño del modelo de embedding de GPT-2 con el de nuestro tokenizer. Si no, el entrenamiento fallará.


In [ ]:
gpt2_base_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50257, 768)

Ahora definimos nuestro `pipeline` de entrenamiento. Y lo probaremos con nuestro modelo sin entrenar


In [ ]:
from transformers import pipeline, set_seed, GenerationConfig

# Inicializamos la pipeline de generación de texto con el modelo y tokenizer específicos de GPT-2.
generator = pipeline('text-generation', model=gpt2_base_model, tokenizer=gpt2_tokenizer)

# Establece una semilla para la generación de números aleatorios, asegurando la reproducibilidad de los resultados.
set_seed(42)

def generate_sample(sample):
  # Prepara el prompt para la generación, incorporando la pregunta y la respuesta SQL esperada.
  prompt_package = create_sql_prompt_and_response(sample)

  # Configuración para la generación de texto, ajustando parámetros como el número máximo de tokens nuevos,
  # la probabilidad de muestreo, entre otros, para controlar la calidad y la variabilidad del texto generado.
  generation_config = GenerationConfig(
      max_new_tokens=50, # Número máximo de tokens nuevos a generar.
      do_sample=True, # Activa el muestreo; si es False, utiliza la predicción más probable en cada paso.
      top_k=50, # El modelo solo considerará los 50 tokens más probables en cada paso de generación.
      temperature=1e-4, # Controla la "creatividad" de las respuestas; valores más bajos hacen que el modelo sea más conservador.
      eos_token_id=gpt2_base_model.config.eos_token_id, # Identificador del token de fin de secuencia, para indicar cuando detener la generación.
  )

  # Genera texto basado en el prompt y la configuración especificada.
  generation = generator(prompt_package["full_prompt"], generation_config=generation_config)
  print("---------------")
  print("Question:")
  print(sample["question"])
  print("---------------")
  print("Dataset Response")
  # Imprime la respuesta esperada para comparación.
  print(prompt_package["ground_truth"])
  print("---------------")
  print("Model Response:")
  # Imprime la respuesta del modelo, eliminando el texto del prompt para mostrar solo el texto generado nuevo.
  print(generation[0]["generated_text"].replace(prompt_package["full_prompt"], ""))


Device set to use cuda:0


In [ ]:
generate_sample(split_sql_dataset["test"][40])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
What university did Steve Hoar attend. 
---------------
Dataset Response
SELECT alma_mater FROM table_18042409_1 WHERE player = "Steve Hoar"
---------------
Model Response:
The following is a list of the most popular and popular games in the world.

The following is a list of the most popular and popular games in the world.

The following is a list of the most popular and popular games in the


El modelo no es capaz de hacer lo que le hemos pedido... Ahora mismo es un modelo fundacional sin ninguna tarea enseñada y no es capaz ni de entender qué le hemos pedido.

En `Ground Truth` tenemos el resultado aproximado que debería dar...


## Entrenamiento del Modelo

Ahora que tenemos configurado nuestro modelo y nuestro tokenizador, finalmente podemos comenzar el entrenamiento!

Utilizaremos los `TrainingArguments` de la biblioteca `transformers` de Hugging Face para ayudarnos a hacer seguimiento de nuestros hiperparámetros. Más información y documentación disponible:

- [`TrainingArguments`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)

Vamos a ver nuestro Trainer y a configurar algunos hiperparámetros:

- `per_device_train_batch_size`: Esta es la cantidad de datos que el modelo procesa en cada iteración de entrenamiento por dispositivo. Ajustar este valor puede ayudar a optimizar el uso de la memoria y la velocidad de entrenamiento. En entrenamiento distribuido, cada dispositivo (por ejemplo, cada GPU) procesará esta cantidad de datos.

- `gradient_accumulation_steps`: Este parámetro permite "simular" un tamaño de lote más grande sin necesidad de más memoria. En lugar de actualizar los pesos del modelo después de cada lote, los gradientes se acumulan durante varios lotes antes de realizar la actualización. Esto puede ser útil cuando el tamaño de lote deseado es más grande de lo que permite la memoria disponible.

- `gradient_checkpointing`: Es una técnica para ahorrar memoria durante el entrenamiento, especialmente útil en modelos muy grandes. Funciona guardando solo ciertos estados intermedios durante el paso hacia adelante y recalculando los estados cuando se necesitan durante el paso hacia atrás. Esto reduce el consumo de memoria a costa de un aumento del tiempo computacional.

- `max_grad_norm`: Este parámetro se utiliza en el recorte de gradientes, que es una técnica para evitar la explosión de gradientes limitando el tamaño de los gradientes. Esto se logra recortando los gradientes de manera que su norma no exceda este valor.

- `max_steps`: Define el número total de pasos de entrenamiento. Un paso de entrenamiento equivale a una actualización de los pesos del modelo después de procesar `per_device_train_batch_size * gradient_accumulation_steps` muestras.

- `learning_rate`: Este es el ritmo con el que el modelo aprende durante el entrenamiento. Valores demasiado altos pueden hacer que el entrenamiento sea inestable, mientras que valores demasiado bajos pueden hacer que el entrenamiento sea muy lento.

- `save_total_limit`: Este parámetro limita el número de versiones del modelo que se guardan durante el entrenamiento. Esto puede ayudar a gestionar el espacio de almacenamiento evitando guardar demasiados archivos de modelo.

- `logging_steps`: Especifica cada cuántos pasos de entrenamiento se registra información como la pérdida de entrenamiento. Esto es útil para hacer seguimiento del progreso del entrenamiento.

- `output_dir`: Este es el directorio donde se guardarán los checkpoints del modelo, así como cualquier otra salida generada durante el entrenamiento.

- `optim`: Define el optimizador que se utilizará para actualizar los pesos del modelo. Los optimizadores controlan cómo se modifican los pesos del modelo en respuesta al gradiente de los errores calculados.

- `lr_scheduler_type`: El programador de la tasa de aprendizaje controla cómo la tasa de aprendizaje cambia a lo largo del tiempo. Un programador cósmico ajusta la tasa de aprendizaje siguiendo una curva que inicialmente disminuye y luego aumenta, similar a la forma de una curva cósmica.

- `evaluation_strategy`: Este parámetro determina cuándo se utiliza el conjunto de datos de evaluación para evaluar el rendimiento del modelo durante el entrenamiento.

- `eval_steps`: Indica cada cuántos pasos de entrenamiento se realiza una evaluación del modelo utilizando el conjunto de datos de evaluación.

- `warmup_ratio`: Este es el porcentaje de los `max_steps` totales durante los cuales la tasa de aprendizaje aumenta gradualmente desde cero hasta la tasa de aprendizaje inicial. Después de este período de calentamiento, la tasa de aprendizaje comienza a disminuir según el programador de la tasa de aprendizaje seleccionado.


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=4,
 gradient_checkpointing =True,
 max_grad_norm= 0.3,
 #num_train_epochs=2,
 max_steps=100,
 learning_rate=2e-4,
 save_total_limit=3,
 logging_steps=10,
 output_dir="sql_gpt2",
 optim="paged_adamw_32bit",
 lr_scheduler_type="cosine",
 #evaluation_strategy="steps",
 eval_steps=50,
 warmup_ratio=0.05,
    report_to="none",
)

###❓Pregunta

¿Qué tipo de entrenamiento estamos siguiendo aquí? ¿Supervisado o no supervisado?




---



Ahora, en cuanto a nuestro `SFTTrainer`, también conocido como "Donde ocurre la magia".

Puedes leer todo sobre el `SFTTrainer` aquí:

- [`SFTTrainer`](https://huggingface.co/docs/trl/sft_trainer#trl.SFTTrainer)

Este `SFTTrainer` tomará los argumentos de entrenamiento que hemos definido antes, nuestros datos, nuestro modelo y nuestro tokenizador, ¡y entrenará todo para nosotros!

Fíjate que estamos estableciendo `max_seq_length` al máximo de ventana de contexto de nuestro modelo - esto asegura que no superemos nuestra máxima ventana de contexto, ¡y rellenará nuestros ejemplos hasta el máximo de la ventana de contexto!


In [ ]:
trainer = SFTTrainer(
 gpt2_base_model,
 train_dataset=split_sql_dataset["train"],
 eval_dataset=split_sql_dataset["val"],
 #tokenizer=gpt2_tokenizer,
 args=training_args
)

Converting train dataset to ChatML:   0%|          | 0/62861 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/62861 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/62861 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/62861 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/7858 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/7858 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/7858 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/7858 [00:00<?, ? examples/s]

### ¡Entrenar!

30 min aprox en una T4


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.407700
20,0.871000
30,0.721000
40,0.740300
50,0.687100
60,0.721400
70,0.691700
80,0.688100
90,0.685600
100,0.670200


TrainOutput(global_step=100, training_loss=0.8884336614608764, metrics={'train_runtime': 109.0198, 'train_samples_per_second': 14.676, 'train_steps_per_second': 0.917, 'total_flos': 132103535616000.0, 'train_loss': 0.8884336614608764})

Guardamos el modelo

In [ ]:
trainer.save_model()

## Probemos el nuevo modelo

Ya tenemos nuestro modelo entrenado en pocas iteraciones, vamos a ver cuál es el resultado...


In [ ]:
ft_gpt2_model = AutoModelForCausalLM.from_pretrained("sql_gpt2")

In [ ]:
generator = pipeline('text-generation', model=ft_gpt2_model, tokenizer=gpt2_tokenizer, )

Device set to use cuda:0


In [ ]:
for i in range(20):
    generate_sample(split_sql_dataset["test"][i])
    print('========================================================')
    print('========================================================')
    print('========================================================')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
What player has a score larger than 67?
---------------
Dataset Response
SELECT player FROM table_name_94 WHERE score > 67
---------------
Model Response:
SELECT player FROM table_name_94 WHERE score = 67


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
What's the rank average when the gold medals are less than 0?
---------------
Dataset Response
SELECT AVG(rank) FROM table_name_41 WHERE gold < 0
---------------
Model Response:
SELECT AVG(rank) FROM table_name_41 WHERE gold > 0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

---------------
Question:
What is the average number of laps when the country was NL and the race number was smaller than 8?
---------------
Dataset Response
SELECT AVG(laps) FROM table_name_96 WHERE race_number < 8 AND country = "nl"
---------------
Model Response:
SELECT AVG(laps) FROM table_name_96 WHERE race_number = 8


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Name the Score of craig stadler?
---------------
Dataset Response
SELECT score FROM table_name_33 WHERE player = "craig stadler"
---------------
Model Response:
SELECT score FROM table_name_33 WHERE player = "Craig Stadler"


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------
Question:
Which Home team has a Year of 2002/03, and a Venue of tahnoun bin mohamed stadium?
---------------
Dataset Response
SELECT home_team FROM table_name_35 WHERE year = "2002/03" AND venue = "tahnoun bin mohamed stadium"
---------------
Model Response:
SELECT home_team FROM table_name_35 WHERE year = "2002/03" AND venue = "tahnoun bin mohamed stadium"
---------------
Question:
Name the least year for les triplettes de belleville
---------------
Dataset Response
SELECT MIN(year) FROM table_name_45 WHERE original_title = "les triplettes de belleville"
---------------
Model Response:
SELECT MIN(year) FROM table_name_45 WHERE original_title = "Les triplettes de belleville"


Mucho mejor, ¿no? 🙂


###❓Pregunta

¿Cómo podrían evaluar su SQL generado? Por favor, proporcionen 3 métodos diferentes.


In [1]:
SELECT home_team FROM table_name_35 WHERE year = "2002/03" AND venue = "tahnoun bin mohamed stadium"
SELECT home_team FROM table_name_35 WHERE venue = "tahnoun bin mohamed stadium" AND year = "2002/03"

SyntaxError: invalid syntax (<ipython-input-1-80d8b1fef4f7>, line 1)

- Linter SQL
- Cotejar con TEST
- Modelo supervisor
- Revisión humana
- Cotejar resultados de la tabla SQL